# Final project: StackOverflow assistant bot

Congratulations on coming this far and solving the programming assignments! In this final project, we will combine everything we have learned about Natural Language Processing to construct a *dialogue chat bot*, which will be able to:
* answer programming-related questions (using StackOverflow dataset);
* chit-chat and simulate dialogue on all non programming-related questions.

For a chit-chat mode we will use a pre-trained neural network engine available from [ChatterBot](https://github.com/gunthercox/ChatterBot).
Those who aim at honor certificates for our course or are just curious, will train their own models for chit-chat.
![](https://imgs.xkcd.com/comics/twitter_bot.png)
©[xkcd](https://xkcd.com)

### Data description

To detect *intent* of users questions we will need two text collections:
- `tagged_posts.tsv` — StackOverflow posts, tagged with one programming language (*positive samples*).
- `dialogues.tsv` — dialogue phrases from movie subtitles (*negative samples*).


In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    ! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
    import setup_google_colab
    setup_google_colab.setup_project()

import sys
sys.path.append("..")
from common.download_utils import download_project_resources

download_project_resources()

File data\dialogues.tsv is already downloaded.
File data\tagged_posts.tsv is already downloaded.


For those questions, that have programming-related intent, we will proceed as follow predict programming language (only one tag per question allowed here) and rank candidates within the tag using embeddings.
For the ranking part, you will need:
- `word_embeddings.tsv` — word embeddings, that you  trained with StarSpace in the 3rd assignment. It's not a problem if you didn't do it, because we can offer an alternative solution for you.

As a result of this notebook, you should obtain the following new objects that you will then use in the running bot:

- `intent_recognizer.pkl` — intent recognition model;
- `tag_classifier.pkl` — programming language classification model;
- `tfidf_vectorizer.pkl` — vectorizer used during training;
- `thread_embeddings_by_tags` — folder with thread embeddings, arranged by tags.
    

Some functions will be reused by this notebook and the scripts, so we put them into *utils.py* file. Don't forget to open it and fill in the gaps!

In [1]:

from utils import *

[nltk_data] Downloading package stopwords to C:\Users\Shafeek
[nltk_data]     Saleem\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
stopwords_set = set(stopwords.words('english'))

## Part I. Intent and language recognition

In [4]:
stopwords_set

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

We want to write a bot, which will not only **answer programming-related questions**, but also will be able to **maintain a dialogue**. We would also like to detect the *intent* of the user from the question (we could have had a 'Question answering mode' check-box in the bot, but it wouldn't fun at all, would it?). So the first thing we need to do is to **distinguish programming-related questions from general ones**.

It would also be good to predict which programming language a particular question referees to. By doing so, we will speed up question search by a factor of the number of languages (10 here), and exercise our *text classification* skill a bit. :)

In [2]:
import numpy as np
import pandas as pd
import pickle
import re
from utils import *
from sklearn.feature_extraction.text import TfidfVectorizer

### Data preparation

In the first assignment (Predict tags on StackOverflow with linear models), you have already learnt how to preprocess texts and do TF-IDF tranformations. Reuse your code here. In addition, you will also need to [dump](https://docs.python.org/3/library/pickle.html#pickle.dump) the TF-IDF vectorizer with pickle to use it later in the running bot.

In [6]:
def tfidf_features(X_train, X_test, vectorizer_path):
    """Performs TF-IDF transformation and dumps the model."""
    
    # Train a vectorizer on X_train data.
    # Transform X_train and X_test data.
    
    # Pickle the trained vectorizer to 'vectorizer_path'
    # Don't forget to open the file in writing bytes mode.
    
    tfidf_vectorizer = TfidfVectorizer(min_df = 5, max_df = 0.9, ngram_range=(1,3))

    tfidf_vectorizer.fit(X_train)
    X_train = tfidf_vectorizer.transform(X_train)
    X_test  = tfidf_vectorizer.transform(X_test)
    
    pickle_out = open(vectorizer_path,"wb")
    pickle.dump(tfidf_vectorizer, pickle_out)
    pickle_out.close()
    return X_train, X_test

Now, load examples of two classes. Use a subsample of stackoverflow data to balance the classes. You will need the full data later.

In [54]:
sample_size = 200000

dialogue_df = pd.read_csv('data/dialogue_cleaned_sw.csv').sample(sample_size, random_state=0)
#dialogue_df_2 = pd.read_csv('data/dialogues.tsv',sep='\t').sample(sample_size, random_state=0)
stackoverflow_df = pd.read_csv('data/stackoverflow_cleaned_sw.csv').sample(sample_size, random_state=0)
say_myname_df = pd.read_csv('data/say_my_name.tsv', sep='\t').sample(2000, random_state=0, replace=True)
ask_builder_df = pd.read_csv('data/ask_who_built.tsv', sep='\t').sample(2000, random_state=0,replace=True)
ask_age_df = pd.read_csv('data/ask_age.tsv', sep='\t').sample(2000, random_state=0,replace=True)
bot_challenge_df = pd.read_csv('data/bot_challenge.tsv', sep='\t').sample(2000, random_state=0,replace=True)



Check how the data look like:

In [4]:
stackoverflow_df = pd.read_csv('data/stackoverflow_cleaned_sw.csv')

In [6]:
stackoverflow_df.shape

(200000, 4)

Apply *text_prepare* function to preprocess the data:

In [56]:
from utils import text_prepare

In [10]:
dialogue_df['text'] = dialogue_df['text'].apply(lambda x: text_prepare(x,remove_stop_words=True))
stackoverflow_df['title'] = stackoverflow_df['title'].apply(lambda x: text_prepare(x,remove_stop_words=True))

In [12]:
dialogue_df.to_csv("data/dialogue_cleaned_sw.csv")

In [13]:
stackoverflow_df.to_csv("data/stackoverflow_cleaned_sw.csv")

In [57]:
say_myname_df['Can you tell me my name?'] = say_myname_df['Can you tell me my name?'].apply(lambda x: text_prepare(x, remove_stop_words=True))
bot_challenge_df['Ar you a bot ?'] = bot_challenge_df['Ar you a bot ?'].apply(lambda x: text_prepare(x,remove_stop_words=True))
ask_builder_df['By whom were you built?'] = ask_builder_df['By whom were you built?'].apply(lambda x: text_prepare(x,remove_stop_words=True))
ask_age_df['Do you have friends the same age as you,'] = ask_age_df['Do you have friends the same age as you,'].apply(lambda x: text_prepare(x,remove_stop_words=True))

In [58]:
ask_age_df.shape[0]

2000

### Intent recognition

We will do a binary classification on TF-IDF representations of texts. Labels will be either `dialogue` for general questions or `stackoverflow` for programming-related questions. First, prepare the data for this task:
- concatenate `dialogue` and `stackoverflow` examples into one sample
- split it into train and test in proportion 9:1, use *random_state=0* for reproducibility
- transform it into TF-IDF features

In [9]:
from sklearn.model_selection import train_test_split

In [60]:
dialogue_df.head()

,Unnamed: 0,text,tag
54458,123831,whites mindless thug,dialogue
118646,64789,listen need worry stuff happened behind us let...,dialogue
57401,130229,dont place much moneys getting,dialogue
115324,190222,got kicked first movie,dialogue
71684,83178,id like ask something four years ago working n...,dialogue


In [16]:
X = np.concatenate([dialogue_df['text'].values, stackoverflow_df['title'].values])
y = ['dialogue'] * dialogue_df.shape[0] + ['stackoverflow'] * stackoverflow_df.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))



Train size = 360000, test size = 40000


In [61]:
X = np.concatenate([dialogue_df['text'].values, stackoverflow_df['title'].values, say_myname_df['Can you tell me my name?'].values,\
                    bot_challenge_df['Ar you a bot ?'].values,ask_builder_df['By whom were you built?'].values,ask_age_df['Do you have friends the same age as you,'].values])
y = ['dialogue'] * dialogue_df.shape[0] + ['stackoverflow'] * stackoverflow_df.shape[0] + ['say_name']*say_myname_df.shape[0]+\
        ['bot_challenge']*bot_challenge_df.shape[0] +['ask_builder']*ask_builder_df.shape[0]+['ask_age']*ask_age_df.shape[0]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.1,random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

Train size = 367200, test size = 40800


In [65]:
X_train = X_train.astype('U')

In [67]:
X_test = X_test.astype('U')

In [68]:
X_train_tfidf, X_test_tfidf = tfidf_features(X_train, X_test, RESOURCE_PATH["TFIDF_VECTORIZER"])

Train the **intent recognizer** using LogisticRegression on the train set with the following parameters: *penalty='l2'*, *C=10*, *random_state=0*. Print out the accuracy on the test set to check whether everything looks good.

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [69]:
intent_recognizer = LogisticRegression(penalty='l2', C=10, random_state=0)
intent_recognizer.fit(X_train_tfidf,y_train)

C:\Users\Shafeek Saleem\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Shafeek Saleem\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [70]:
# Check test accuracy.
y_test_pred = intent_recognizer.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.9908578431372549


In [71]:
vectorizer = pickle.load(open(RESOURCE_PATH['TFIDF_VECTORIZER'], 'rb'))

In [94]:
k = 'how to convert an array to list'
k = text_prepare(k,remove_stop_words=True)
k = vectorizer.transform(np.array([k],dtype=object))

In [95]:
intent_recognizer.predict(k)

array(['stackoverflow'], dtype='<U13')

In [7]:
vectorizer = pickle.load(open(RESOURCE_PATH['TFIDF_VECTORIZER'], 'rb'))

In [83]:
intent_recognizer = pickle.load(open(RESOURCE_PATH['INTENT_RECOGNIZER'], 'rb'))


Dump the classifier to use it in the running bot.

In [96]:
pickle.dump(intent_recognizer, open(RESOURCE_PATH['INTENT_RECOGNIZER'], 'wb'))

### Programming language classification 

We will train one more classifier for the programming-related questions. It will predict exactly one tag (=programming language) and will be also based on Logistic Regression with TF-IDF features. 

First, let us prepare the data for this task.

In [10]:
X = stackoverflow_df['title'].values
y = stackoverflow_df['tag'].values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print('Train size = {}, test size = {}'.format(len(X_train), len(X_test)))

Train size = 160000, test size = 40000


Let us reuse the TF-IDF vectorizer that we have already created above. It should not make a huge difference which data was used to train it.

In [13]:
vectorizer = pickle.load(open(RESOURCE_PATH['TFIDF_VECTORIZER'], 'rb'))

X_train_tfidf, X_test_tfidf = vectorizer.transform(X_train.astype('U')), vectorizer.transform(X_test.astype('U'))

In [17]:
k=np.array([X_train[10]],dtype=object)

In [18]:
X_train[10]

'get distinct sum php array'

In [19]:
k = vectorizer.transform(k)

Train the **tag classifier** using OneVsRestClassifier wrapper over LogisticRegression. Use the following parameters: *penalty='l2'*, *C=5*, *random_state=0*.

In [14]:
from sklearn.multiclass import OneVsRestClassifier

In [15]:
tag_classifier = OneVsRestClassifier(LogisticRegression(penalty='l2',max_iter=500,C=10,random_state=42)).fit(X_train_tfidf, y_train)

C:\Users\Shafeek Saleem\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [16]:
# Check test accuracy.
y_test_pred = tag_classifier.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print('Test accuracy = {}'.format(test_accuracy))

Test accuracy = 0.77065


In [20]:
tag_classifier.predict(k)

array(['php'], dtype='<U10')

Dump the classifier to use it in the running bot.

In [21]:
pickle.dump(tag_classifier, open(RESOURCE_PATH['TAG_CLASSIFIER'], 'wb'))

In [3]:
import gensim

In [5]:
def load_embeddings(embeddings_path):
    """Loads pre-trained word embeddings from tsv file.

    Args:
      embeddings_path - path to the embeddings file.

    Returns:
      embeddings - dict mapping words to vectors;
      embeddings_dim - dimension of the vectors.
    """

    # Hint: you have already implemented a similar routine in the 3rd assignment.
    # Note that here you also need to know the dimension of the loaded embeddings.
    # When you load the embeddings, use numpy.float32 type as dtype
    wv_embeddings = gensim.models.KeyedVectors.load_word2vec_format(embeddings_path,binary=True)
    embeddings_dim = 50
    embeddings = {}
    for word in wv_embeddings.vocab:
        embeddings[word] = wv_embeddings[word]
    return embeddings, embeddings_dim

## Part II. Ranking  questions with embeddings

To find a relevant answer (a thread from StackOverflow) on a question you will use vector representations to calculate similarity between the question and existing threads. We already had `question_to_vec` function from the assignment 3, which can create such a representation based on word vectors. 

However, it would be costly to compute such a representation for all possible answers in *online mode* of the bot (e.g. when bot is running and answering questions from many users). This is the reason why you will create a *database* with pre-computed representations. These representations will be arranged by non-overlaping tags (programming languages), so that the search of the answer can be performed only within one tag each time. This will make our bot even more efficient and allow not to store all the database in RAM. 

Load StarSpace embeddings which were trained on Stack Overflow posts. These embeddings were trained in *supervised mode* for duplicates detection on the same corpus that is used in search. We can account on that these representations will allow us to find closely related answers for a question. 

If for some reasons you didn't train StarSpace embeddings in the assignment 3, you can use [pre-trained word vectors](https://code.google.com/archive/p/word2vec/) from Google. All instructions about how to work with these vectors were provided in the same assignment. However, we highly recommend to use StarSpace's embeddings, because it contains more appropriate embeddings. If you chose to use Google's embeddings, delete the words, which is not in Stackoverflow data.

In [6]:
wv_embeddings, embeddings_dim = load_embeddings("word2vec50.bin")

In [7]:
embeddings_dim

50

In [26]:
wv_embeddings_ = gensim.models.KeyedVectors.load_word2vec_format("word2vec50.bin",binary=True)

In [27]:
k = np.zeros(100)
k+=wv_embeddings['python']

In [29]:
k.shape

(100,)

Since we want to precompute representations for all possible answers, we need to load the whole posts dataset, unlike we did for the intent classifier:

In [8]:
posts_df = pd.read_csv('data/tagged_posts.tsv', sep='\t')

In [9]:
posts_df.head()

,post_id,title,tag
0,9,Calculate age in C#,c#
1,16,Filling a DataSet or DataTable from a LINQ que...,c#
2,39,Reliable timer in a console application,c#
3,42,Best way to allow plugins for a PHP application,php
4,59,"How do I get a distinct, ordered list of names...",c#


Look at the distribution of posts for programming languages (tags) and find the most common ones. 
You might want to use pandas [groupby](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html) and [count](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.count.html) methods:

In [10]:
counts_by_tag = posts_df.groupby('tag').count()

In [11]:
title_counts = [a for a in counts_by_tag['title'].values]

In [12]:
tags =  list(counts_by_tag.index)

In [13]:
count_by_tag_list = list(zip(tags,title_counts))

In [14]:
for a,b in count_by_tag_list:
    print(a,b)

c# 394451
c_cpp 281300
java 383456
javascript 375867
php 321752
python 208607
r 36359
ruby 99930
swift 34809
vb 35044


Now for each `tag` you need to create two data structures, which will serve as online search index:
* `tag_post_ids` — a list of post_ids with shape `(counts_by_tag[tag],)`. It will be needed to show the title and link to the thread;
* `tag_vectors` — a matrix with shape `(counts_by_tag[tag], embeddings_dim)` where embeddings for each answer are stored.

Implement the code which will calculate the mentioned structures and dump it to files. It should take several minutes to compute it.

In [15]:
import os
os.makedirs(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], exist_ok=True)

for tag, count in count_by_tag_list:
    tag_posts = posts_df[posts_df['tag'] == tag]
    tag_post_ids = tag_posts['post_id'].to_list()
    
    tag_vectors = np.zeros((count, embeddings_dim), dtype=np.float32)
    for i, title in enumerate(tag_posts['title']):
        tag_vectors[i, :] = question_to_vec(title, wv_embeddings, embeddings_dim)

    # Dump post ids and vectors to a file.
    filename = os.path.join(RESOURCE_PATH['THREAD_EMBEDDINGS_FOLDER'], os.path.normpath('%s.pkl' % tag))
    pickle.dump((tag_post_ids, tag_vectors), open(filename, 'wb'))

In [1]:
k = [1.3,5,6,8]

In [2]:
import random

In [22]:
random.choice(k)

1.3

In [ ]:
thread_ids, thread_embeddings = unpickle_file(embeddings_path)